<a href="https://colab.research.google.com/github/xavirubi/tappx-challenge/blob/master/hiring_project/kw_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U keybert

In [ ]:
!pip install -U rake-nltk

In [ ]:
!pip install -U yake

In [ ]:
!pip install -U gensim

In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download es_core_news_lg

In [ ]:
import json
# from keybert import KeyBERT
# from rake_nltk import Rake
import nltk
# import yake
from gensim.models import Word2Vec

nltk.download('stopwords')
nltk.download('punkt')

In [7]:
f_articles = open('articles.json')
f_videos = open('videos.json')

# articles_data = json.load(f_articles)
# videos_data = json.load(f_videos)

# article_ids = list(article_id for article_id in articles_data.keys())
# video_ids = list(video_id for video_id in videos_data.keys())

In [ ]:
def article_text(id):
	return articles_data[id]["text"]

def video_text(id):
	return videos_data[id]["text"]

def article_keywords(id):
	return articles_data[id]["keywords"]

def video_keywords(id):
	return videos_data[id]["keywords"]

In [ ]:
kw_model = KeyBERT()
def keybert_keywords(text):
  return kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 3))

In [ ]:
rake_model = Rake()
def rake_keywords(text):
  kw_extracted = rake_model.extract_keywords_from_text(text)
  return rake_model.get_ranked_phrases_with_scores()

In [ ]:
def yake_keywords(text):
  kw_extractor = yake.KeywordExtractor(lan='en', n=3)
  return kw_extractor.extract_keywords(text)

In [ ]:
for id in article_ids:
  print(f'''TEXT FROM ARTICLE {id}:
  {article_text(id)}
''')
  print(f'''KEYWORDS FROM ARTICLE {id}:
  {article_keywords(id)}
''')
  print(f'''KEYBERT KEYWORDS FROM ARTICLE {id}:
{keybert_keywords(article_text(id))}
''')

  print(f'''RAKE KEYWORDS FROM ARTICLE {id}:
{rake_keywords(article_text(id))}
''')

  print(f'''YAKE KEYWORDS FROM ARTICLE {id}:
{yake_keywords(article_text(id))}



-----------------------------------------------------------------------------------------------------
''')

f_articles.close()

In [ ]:
corpus1 = article_keywords(article_ids[0])
# corpus2 = article_keywords(article_ids[7])
# print(f"CORPUS 1: {corpus1}\n\nCORPUS 2: {corpus2}\n--------------------------------------------------------------\n\n")

model = Word2Vec(corpus1, window=5, min_count=1, workers=4)

model = Word2Vec(corpus1)

In [ ]:
for i in range(0, len(video_ids)):
  corpus2 = video_keywords(video_ids[i])
  similarity_score = model.wv.n_similarity(corpus1, corpus2)
  print(f"SIMILARITY SCORE: {similarity_score}\n")

In [ ]:
import spacy

nlp = spacy.load("es_core_news_lg")
doc1 = nlp(article_text(article_ids[1]))
# doc2 = nlp(video_text(video_ids[0]))
print(article_keywords(article_ids[1]))
# Similarity of two documents
for i in range(0, len(video_ids)):
  doc2 = nlp(video_text(video_ids[i]))
  print(i, doc1.similarity(doc2), video_keywords(video_ids[i]))


In [ ]:
print(article_keywords(article_ids[0]))

def score_func(article_kws, video_kws):
  score = 0
  for article_kw in article_kws:
    for video_kw in video_kws:
      score += nlp(article_kw).similarity(nlp(video_kw))
  return score

for i in range(0, len(video_ids)):
  score = score_func(article_keywords(article_ids[0]), video_keywords(video_ids[i]))
  print(i, score, video_keywords(video_ids[i]), end='\n\n')


In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")


In [ ]:
candidate_labels = article_keywords(article_ids[0])

for i in range(0, len(video_ids)):
  sequence_to_classify = video_text(video_ids[i])
  print(classifier(sequence_to_classify, candidate_labels, multi_label=True))

#{'labels': ['travel', 'exploration', 'dancing', 'cooking'],
# 'scores': [0.9945111274719238,
#  0.9383890628814697,
#  0.0057061901316046715,
#  0.0018193122232332826],
# 'sequence': 'one day I will see the world'}


In [ ]:
!pip install scikit-learn

In [9]:
import json
from collections import defaultdict
from typing import List, Dict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [10]:
def article_keywords_to_string(article: Dict) -> str:
    return " ".join(article['keywords'])

def video_keywords_to_string(video: Dict) -> str:
    return " ".join(video['keywords'])

In [16]:
def assign_videos_to_articles(articles: List[Dict], videos: List[Dict], threshold: float = 0.3) -> Dict[int, List[int]]:
    article_keywords = [article_keywords_to_string(article) for article in articles]
    video_keywords = [video_keywords_to_string(video) for video in videos]

    vectorizer = TfidfVectorizer()
    article_matrix = vectorizer.fit_transform(article_keywords)
    video_matrix = vectorizer.transform(video_keywords)

    similarity_matrix = cosine_similarity(article_matrix, video_matrix)

    related_videos = defaultdict(list)
    for article_index, article_row in enumerate(similarity_matrix):
      print(f"article index: {article_index} , article row: {article_row}")
      for video_index, similarity_score in enumerate(article_row):
          if similarity_score >= threshold:
              related_videos[article_index].append(video_index)

    return related_videos

In [17]:
f_articles = open('articles.json')
f_videos = open('videos.json')
article_data = json.load(f_articles)
video_data = json.load(f_videos)

list_of_articles = list(value for value in article_data.values())
list_of_videos = list(value for value in video_data.values())

related_videos = assign_videos_to_articles(list_of_articles, list_of_videos)

for article_index, video_indices in related_videos.items():
    print(f"Article {article_index}: {list_of_articles[article_index]['keywords']}")
    for video_index in video_indices:
      print(f"  Related Video: {list_of_videos[video_index]['keywords']}")


article index: 0 , article row: [0.12207095 0.22226155 0.13258306 0.0547024  0.16233808 0.01933188
 0.21553044 0.11410904 0.04668448 0.26581964 0.1623422  0.16167335
 0.         0.1414228  0.22532006 0.09573568 0.03878312 0.21246275
 0.14159545 0.28647039 0.14364496 0.07362294 0.         0.14783387
 0.15635901 0.27076934 0.14934092 0.24510633 0.18482325 0.2146687
 0.07362294 0.07790366 0.22368271 0.10927759 0.         0.2073179
 0.         0.22011852 0.22381001]
article index: 1 , article row: [0.14115643 0.25701158 0.15331209 0.15269913 0.26603005 0.08557365
 0.24922808 0.1319497  0.20665149 0.35724588 0.11812653 0.099243
 0.         0.1635339  0.29284621 0.11070371 0.04484676 0.24568077
 0.16373354 0.33125931 0.16610349 0.0851337  0.         0.17094732
 0.18080534 0.19702236 0.17268999 0.28342808 0.21371989 0.18339893
 0.0851337  0.13254392 0.16276029 0.12636287 0.         0.20372287
 0.         0.20884117 0.25880214]
article index: 2 , article row: [0.08022144 0.0781464  0.04661575 